In [1]:
# 라이브러리(남이 짜둔 코드) 임포트
import sys
import os
import pandas as pd   # 데이터 분석
import numpy as np    # 행렬, 숫자

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import tqdm
from tqdm.notebook import tqdm

# 검색조건 세팅

In [2]:
keyword = "섬유유연제"

In [3]:
#Step 1. 크롬 웹브라우저 실행
path = "/Users/aiden/Desktop/chromedriver"

driver = webdriver.Chrome(path)
# 사이트 주소는 네이버
driver.get('https://www.lotteon.com/p/display/main/lotteon?mall_no=1')
time.sleep(3)


In [4]:
# 검색어 입력
element = driver.find_element_by_id("headerSearchId")
# element.send_keys(keyword) 

for c in keyword:
    element.send_keys(c)
    time.sleep(0.2)
    
# 검색버튼 클릭
driver.find_element_by_class_name("btnSearchInner").click()
time.sleep(2)

In [5]:
# 리뷰 많은 순 클릭
srch_options = driver.find_elements_by_css_selector(".srchResultSortOptions .srchFilterOption")
srch_options[2].click()

# 상품별 url 수집

In [6]:
url = ".srchProductUnitImageArea > a"
url_raws = driver.find_elements_by_css_selector(url)

url_list = []
for url in url_raws:
    url_raw = url.get_attribute('href')
    url_list.append(url_raw)

print(len(url_list))    
url_list

141


['https://www.lotteon.com/p/product/LM8801101168123?sitmNo=LM8801101168123_001&mall_no=1&dp_infw_cd=SCH%EC%84%AC%EC%9C%A0%EC%9C%A0%EC%97%B0%EC%A0%9C',
 'https://www.lotteon.com/p/product/LM8801101168376?sitmNo=LM8801101168376_001&mall_no=1&dp_infw_cd=SCH%EC%84%AC%EC%9C%A0%EC%9C%A0%EC%97%B0%EC%A0%9C',
 'https://www.lotteon.com/p/product/LM8801101168246?sitmNo=LM8801101168246_001&mall_no=1&dp_infw_cd=SCH%EC%84%AC%EC%9C%A0%EC%9C%A0%EC%97%B0%EC%A0%9C',
 'https://www.lotteon.com/p/product/LM4902430543187?sitmNo=LM4902430543187_001&mall_no=1&dp_infw_cd=SCH%EC%84%AC%EC%9C%A0%EC%9C%A0%EC%97%B0%EC%A0%9C',
 'https://www.lotteon.com/p/product/LM0037000312208?sitmNo=LM0037000312208_001&mall_no=1&dp_infw_cd=SCH%EC%84%AC%EC%9C%A0%EC%9C%A0%EC%97%B0%EC%A0%9C',
 'https://www.lotteon.com/p/product/LO1001200865?sitmNo=LO1001200865_1001200866&mall_no=1&dp_infw_cd=SCH%EC%84%AC%EC%9C%A0%EC%9C%A0%EC%97%B0%EC%A0%9C',
 'https://www.lotteon.com/p/product/LM8801092516774?sitmNo=LM8801092516774_001&mall_no=1&dp_i

In [7]:
df = pd.DataFrame({'url':url_list})
df

,url
0,https://www.lotteon.com/p/product/LM8801101168...
1,https://www.lotteon.com/p/product/LM8801101168...
2,https://www.lotteon.com/p/product/LM8801101168...
3,https://www.lotteon.com/p/product/LM4902430543...
4,https://www.lotteon.com/p/product/LM0037000312...
...,...
136,https://www.lotteon.com/p/product/LO1001208564...
137,https://www.lotteon.com/p/product/LM8801051275...
138,https://www.lotteon.com/p/product/PD47521?anch...
139,https://www.lotteon.com/p/product/LM8801051282...


In [8]:
df['url'][0]

'https://www.lotteon.com/p/product/LM8801101168123?sitmNo=LM8801101168123_001&mall_no=1&dp_infw_cd=SCH%EC%84%AC%EC%9C%A0%EC%9C%A0%EC%97%B0%EC%A0%9C'

In [9]:
# 저장하기
df.to_excel("soft_url.xlsx")

# 제품 리뷰 크롤링하기

In [10]:
# url_list 불러오기
url_load = pd.read_excel("soft_url.xlsx")        # 기본 모델

num_list = len(url_load)

print(num_list)
url_load

141


,Unnamed: 0,url
0,0,https://www.lotteon.com/p/product/LM8801101168...
1,1,https://www.lotteon.com/p/product/LM8801101168...
2,2,https://www.lotteon.com/p/product/LM8801101168...
3,3,https://www.lotteon.com/p/product/LM4902430543...
4,4,https://www.lotteon.com/p/product/LM0037000312...
...,...,...
136,136,https://www.lotteon.com/p/product/LO1001208564...
137,137,https://www.lotteon.com/p/product/LM8801051275...
138,138,https://www.lotteon.com/p/product/PD47521?anch...
139,139,https://www.lotteon.com/p/product/LM8801051282...


In [11]:
# 글 띄우기
url = url_load['url'][0]
driver = webdriver.Chrome("/Users/aiden/Desktop/chromedriver")
driver.get(url)   # 글 띄우기
time.sleep(2)

In [12]:
# 리뷰 버튼 클릭
driver.find_element_by_css_selector(".count").click()
time.sleep(2)

In [13]:
# 리뷰 페이지 번호
pageno_raw = driver.find_element_by_xpath("""//*[@id="reviewMain"]/div[8]/div[1]/div/a[9]/span""")
pageno = int(pageno_raw.text)
pageno

302

In [14]:
review_list = []
user_list = []
date_list = []

In [15]:
i=0
while i < pageno:
    i=i+1
    try:
        # review 수집하기
        overlays1 = ".texting"
        reviews = driver.find_elements_by_css_selector(overlays1)
        
        for review in reviews:    
            review = review.text
            review_list.append(review)

        #     print(len(review_list))
        review_list

#         # userID 수집하기
#         overlays1 = ".userName"
#         users = driver.find_elements_by_css_selector(overlays1)

#         for user in users:    
#             user = user.text
#             user_list.append(user)

#         #     print(len(user_list))
#         user_list

#         # date 수집하기

#         overlays1 = ".date"
#         dates = driver.find_elements_by_css_selector(overlays1)

#         for date in dates:
#             date = date.text
#             date_list.append(date)

#         #     print(len(date_list))
#         date_list

        # 다음장 클릭
        driver.find_element_by_css_selector(".next").click()
        time.sleep(1)
    except:
        continue

    print(len(review_list))
#     print(len(user_list))
#     print(len(date_list))

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100
105
110
115
120
125
130
135
140
145
150
155
160
165
170
175
180
185
190
195
200
205
210
215
220
225
230
235
240
245
250
255
260
265
270
275
280
285
290
295
300
305
310
315
320
325
330
335
340
345
350
355
360
365
370
375
380
385
390
395
400
405
410
415
420
425
430
435
440
445
450
455
460
465
470
475
480
485
490
495
500
505
510
515
520
525
530
535
540
545
550
555
560
565
570
575
580
585
590
595
600
605
610
615
620
625
630
635
640
645
650
655
660
665
670
675
680
685
690
695
700
705
710
715
720
725
730
735
740
745
750
755
760
764
769
774
779
784
789
794
799
804
809
814
819
824
829
834
839
844
849
854
859
864
869
874
879
884
889
894
899
904
909
914
919
924
928
933
938
943
948
953
958
963
968
973
978
983
988
993
998
1003
1008
1013
1018
1023
1028
1033
1038
1043
1048
1053
1058
1063
1068
1073
1078
1083
1088
1093
1098
1103
1108
1113
1118
1123
1128
1133
1138
1143
1148
1153
1158
1163
1168
1173
1178
1183
1188
1193
1198
1203
1208
1213
1218


In [16]:
# 판다스로 만들기
result = pd.DataFrame({'review':review_list})
result

,review
0,핑크로즈는 언제나 좋아요.
1,바로 교환처리해 주셔서 좋았어요.
2,항상 구매하는 제품 입니다
3,좋습니다. 좋습니다. 맛있습니다.
4,자주 애용하고 있어요~또 구매할께요.
...,...
1503,가격할인에 향도 좋아요
1504,늘 쓰던 제품 잘 받았습니다.
1505,완전 좋아요 향이 젤 맘에 들어요 평소보다 2천원정도 저렴하게 산듯해요 ㅋ
1506,조하요 저렴하게 구입해 좋아요


In [17]:
result.to_excel('soft_review.xlsx')